# Objective

The **primary objectives of this project are as follows:

* Use Time Series Analysis to **forecast the number of COVID-19 positive patients in the 5 most affected states** in the coming months.  

* Determine **which models are better suited** for making these predictions: **ARIMA** or **Long Short-Term Mermory** (LSTM) **Recurrent Neural Networks** (RNNs).

The **secondary objective**:

* Find any **social circumstances** that could **contribute** to increase in cases.

# Obtaining Data

COVID-19 is a **highly contagious respiratory infection**.  **Symptoms vary by type and severity** depending on immune system strength and age.  Despite this, most people **generally report flu-like symptoms**.  While contracting the illness can happen as remotely as contact with an object touched by a person with the sickness, the **primary mode of transmission** is from **person-to-person** contact, as well as by **airborne germs**. 

While everyone is at risk of contracting the illness, it is primarily the elderly and people with compromised immune systems that are at risk of death.

## Importing Libraries and Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### Notes on the Data:

So we'll be looking at a few different datasets for different tasks.  This is simply because some data sets are formatted in a way that is better for Time Series Analysis, while others are more suited for Exploratory Data Analysis. 

With that, the data that we **preprocess** will be from the sets we'll be using for **Time Series Analysis**, while the rest will be used **only for our EDA**.

# Preprocessing

First, let's take a look at the ```us_daily.csv```.

## US_Daily.csv
This dataset contains a lot of information on the **national level starting from 1/21/2020 through 9/27/2020** (the day the data was downloaded).  Some of the data includes the **total number of positive cases to-date**, the amount of **tests performed daily**, the **number** of people who **tested negative or positive on a daily basis, etc..**  It makes for a great summary and can serve as a "downsampled" dataset for our models.

In [2]:
us_daily = pd.read_csv('csv_files/us_daily.csv')
us_daily.head()

,date,states,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,totalTestResults,lastModified,total,posNeg,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease,hash
0,20200927,56,7080459,90648092,11136.0,29432.0,404083.0,6080.0,20049.0,1511.0,...,101298794,2020-09-27T00:00:00Z,0,0,307,758,665609,35289,806258,e7c64e674bfc2af02802153452e53628d44c241c
1,20200926,56,7045170,89982483,11183.0,29554.0,403325.0,6057.0,20002.0,1509.0,...,100492536,2020-09-26T00:00:00Z,0,0,866,1154,886140,47733,1004261,e98f5076c72de4a27a283d22756b7d0b9a44d41f
2,20200925,56,6997437,89096343,10905.0,29769.0,402171.0,6133.0,19919.0,1506.0,...,99488275,2020-09-25T00:00:00Z,0,0,844,1331,856519,55526,1011675,8d311e73fe038522a1a6be4bc3202de206ec0adb
3,20200924,56,6941911,88239824,12008.0,30043.0,400840.0,6168.0,19555.0,1560.0,...,98476600,2020-09-24T00:00:00Z,0,0,921,1588,823449,43772,940353,375a88dd29991abc1946cd7f98f4f20a9e37fb5d
4,20200923,56,6898139,87416375,10535.0,29905.0,399252.0,6113.0,19452.0,1544.0,...,97536247,2020-09-23T00:00:00Z,0,0,1157,1451,800878,38567,923704,b4fe7067370631b26f8e988fd2524b5691235a09


In the notebook `"csv_files/CSV_previews"`, we established that the column `states` **does not refer to a state's FIPS code**, but to the **number of states/territories that had patients who tested positive for COVID-19**. 

Let's checkout all 25 columns of the dataset and see what will be relevant to our time series modeling. 

In [3]:
us_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      250 non-null    int64  
 1   states                    250 non-null    int64  
 2   positive                  250 non-null    int64  
 3   negative                  250 non-null    int64  
 4   pending                   208 non-null    float64
 5   hospitalizedCurrently     195 non-null    float64
 6   hospitalizedCumulative    208 non-null    float64
 7   inIcuCurrently            186 non-null    float64
 8   inIcuCumulative           187 non-null    float64
 9   onVentilatorCurrently     187 non-null    float64
 10  onVentilatorCumulative    180 non-null    float64
 11  recovered                 187 non-null    float64
 12  dateChecked               250 non-null    object 
 13  death                     231 non-null    float64
 14  hospitaliz

### Dropping columns

Ok, so we're only interested in the number of positive and negative cases, the number of tests, and possibly the number of deaths.  We can probably use information on the number of states to track how fast the virus spread and maybe even use it to discover where it started in the US.

With that, let's make a new data frame and drop the irrelevant data.

In [4]:
usd_df = us_daily[['date', 'states', 'positive', 'negative', 'death', 
                   'totalTestResults', 'total', 'posNeg', 'deathIncrease', 
                   'negativeIncrease', 'positiveIncrease', 
                   'totalTestResultsIncrease']]
usd_df.head()

,date,states,positive,negative,death,totalTestResults,total,posNeg,deathIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
0,20200927,56,7080459,90648092,196869.0,101298794,0,0,307,665609,35289,806258
1,20200926,56,7045170,89982483,196562.0,100492536,0,0,866,886140,47733,1004261
2,20200925,56,6997437,89096343,195696.0,99488275,0,0,844,856519,55526,1011675
3,20200924,56,6941911,88239824,194852.0,98476600,0,0,921,823449,43772,940353
4,20200923,56,6898139,87416375,193931.0,97536247,0,0,1157,800878,38567,923704


Alright! Before we tackle the issue of the date being an integer and not in date-time, let's look at the values inside the columns `total` and `posNeg` and see if they are really only zeros.

In [5]:
print(f'total value counts: {usd_df.total.value_counts()}')
print(f'posNeg value counts: {usd_df.posNeg.value_counts()}')

total value counts: 0    250
Name: total, dtype: int64
posNeg value counts: 0    250
Name: posNeg, dtype: int64


Ok! Looks like the curators of this dataset decided to make different columns for this information and didn't drop these ones, so let's take care of that.

In [6]:
usdf = usd_df.drop(['total', 'posNeg'], axis=1)
usdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      250 non-null    int64  
 1   states                    250 non-null    int64  
 2   positive                  250 non-null    int64  
 3   negative                  250 non-null    int64  
 4   death                     231 non-null    float64
 5   totalTestResults          250 non-null    int64  
 6   deathIncrease             250 non-null    int64  
 7   negativeIncrease          250 non-null    int64  
 8   positiveIncrease          250 non-null    int64  
 9   totalTestResultsIncrease  250 non-null    int64  
dtypes: float64(1), int64(9)
memory usage: 19.7 KB


### Missing Values

We're missing some data in the column `death`. Let's find out why!

In [7]:
usdf['death'].isna().value_counts()

False    231
True      19
Name: death, dtype: int64

In [8]:
death = usdf[usdf['death'].isna() == True]
death

,date,states,positive,negative,death,totalTestResults,deathIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
231,20200209,2,0,0,NaN,11,0,0,0,0
232,20200208,2,0,0,NaN,11,0,0,0,1
233,20200207,2,0,0,NaN,10,0,0,0,0
234,20200206,2,0,0,NaN,10,0,0,0,1
235,20200205,2,0,0,NaN,9,0,0,0,0
236,20200204,2,0,0,NaN,9,0,0,0,2
237,20200203,2,0,0,NaN,7,0,0,0,3
238,20200202,2,0,0,NaN,4,0,0,0,0
239,20200201,2,0,0,NaN,4,0,0,0,0
240,20200131,2,0,0,NaN,4,0,0,0,0


Ok! It appears that all of these `NaN` values are at the **start of the pandemic**.  Since the people who compiled this data were **certain enough** to say that the total number of **positive cases were zero**, we can be **comfortable with imputing these missing values with zeros** as well.   

In [9]:
usdf = usdf.fillna(value=0)
usdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      250 non-null    int64  
 1   states                    250 non-null    int64  
 2   positive                  250 non-null    int64  
 3   negative                  250 non-null    int64  
 4   death                     250 non-null    float64
 5   totalTestResults          250 non-null    int64  
 6   deathIncrease             250 non-null    int64  
 7   negativeIncrease          250 non-null    int64  
 8   positiveIncrease          250 non-null    int64  
 9   totalTestResultsIncrease  250 non-null    int64  
dtypes: float64(1), int64(9)
memory usage: 19.7 KB


### Fixing Dates

Alright, now for the addressing the data type issue.  `date` is in an integer form, so let's fix that.

In [10]:
usdf['date'] = pd.to_datetime(usdf['date'], format='%Y%m%d')
usdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   date                      250 non-null    datetime64[ns]
 1   states                    250 non-null    int64         
 2   positive                  250 non-null    int64         
 3   negative                  250 non-null    int64         
 4   death                     250 non-null    float64       
 5   totalTestResults          250 non-null    int64         
 6   deathIncrease             250 non-null    int64         
 7   negativeIncrease          250 non-null    int64         
 8   positiveIncrease          250 non-null    int64         
 9   totalTestResultsIncrease  250 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(8)
memory usage: 19.7 KB


In [11]:
usdf.head()

,date,states,positive,negative,death,totalTestResults,deathIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
0,2020-09-27,56,7080459,90648092,196869.0,101298794,307,665609,35289,806258
1,2020-09-26,56,7045170,89982483,196562.0,100492536,866,886140,47733,1004261
2,2020-09-25,56,6997437,89096343,195696.0,99488275,844,856519,55526,1011675
3,2020-09-24,56,6941911,88239824,194852.0,98476600,921,823449,43772,940353
4,2020-09-23,56,6898139,87416375,193931.0,97536247,1157,800878,38567,923704


### Export to CSV
Awesome! Now we'll export this as a new CSV!

In [12]:
usdf.to_csv('us_daily_preprocessed.csv')

## JHU Confirmed

This dataset is one of many from Johns Hopkins University datasets on the COVID-19 pandemic. It contains the number of cases per state and county levels, on a day-by-day basis.  Let's open it up!

In [13]:
jhu_conf = pd.read_csv('csv_files/jhu_confirmed.csv')
jhu_conf.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1673,1690,1691,1714,1715,1738,1757,1764,1773,1785
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,5047,5061,5087,5124,5141,5165,5456,5477,5526,5588
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,830,835,838,848,851,857,873,882,885,886
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,628,632,636,635,638,642,652,654,656,657
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,1542,1551,1560,1573,1580,1594,1608,1611,1617,1618


In [14]:
jhu_conf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3340 entries, 0 to 3339
Columns: 262 entries, UID to 9/28/20
dtypes: float64(3), int64(253), object(6)
memory usage: 6.7+ MB


As you can see, we have a **very large dataframe** stretching out in every direction.  In addition to **262 columns** (most of which are dates which need to be in the index instead of the columns), there are **3300 rows** containing all of the states.  These need to be **split up by counties** and **consolidated** if we're going to get any meaningful visualizations.

Let's first get started with the columns and **drop information that isn't useful**.  Unlike the previous dataset, it will be easier to drop the columns we don't want rather than explicitly name the ones we want to keep. 

Before we do that, let's just double check that this ONLY contains data from the United States.

In [15]:
unassigned = jhu_conf[jhu_conf['Admin2'] == 'Unassigned']
unassigned.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20
64,84090001,US,USA,840,90001.0,Unassigned,Alabama,US,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
95,84090002,US,USA,840,90002.0,Unassigned,Alaska,US,0.0,0.0,...,1,1,1,1,17,17,18,19,20,20
115,84090004,US,USA,840,90004.0,Unassigned,Arizona,US,0.0,0.0,...,3,4,1,1,0,3,0,2,0,1
188,84090005,US,USA,840,90005.0,Unassigned,Arkansas,US,0.0,0.0,...,1649,1691,1686,1592,1624,1778,1766,1857,1772,1900
251,84090006,US,USA,840,90006.0,Unassigned,California,US,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Checking the 3 letter abbreviations for countries
jhu_conf.iso3.unique()

array(['USA', 'ASM', 'GUM', 'MNP', 'PRI', 'VIR'], dtype=object)

In [17]:
# Checking Country_Region
jhu_conf.Country_Region.unique()

array(['US'], dtype=object)

So in addition to the 50 states, we **also have information from the US territories American Samoa** (ASM), **Guam** (GUM), the **Northern Mariana Islands** (MNP), **Puerto Rico** (PRI), and the **Virgin Islands** (VIR).  Although these are territories, they are still part of America, so we'll keep them.  The only issue that could come up is that the column `Admin2` (reserved for county names) will likely contain NaN values. We'll look into this after trimming the columns a bit.

In [18]:
jhu_df = jhu_conf.drop(['UID', 'iso2', 'code3', 'FIPS', 'Country_Region', 
                        'Lat', 'Long_'], axis=1)
jhu_df.head()

,iso3,Admin2,Province_State,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20
0,USA,Autauga,Alabama,"Autauga, Alabama, US",0,0,0,0,0,0,...,1673,1690,1691,1714,1715,1738,1757,1764,1773,1785
1,USA,Baldwin,Alabama,"Baldwin, Alabama, US",0,0,0,0,0,0,...,5047,5061,5087,5124,5141,5165,5456,5477,5526,5588
2,USA,Barbour,Alabama,"Barbour, Alabama, US",0,0,0,0,0,0,...,830,835,838,848,851,857,873,882,885,886
3,USA,Bibb,Alabama,"Bibb, Alabama, US",0,0,0,0,0,0,...,628,632,636,635,638,642,652,654,656,657
4,USA,Blount,Alabama,"Blount, Alabama, US",0,0,0,0,0,0,...,1542,1551,1560,1573,1580,1594,1608,1611,1617,1618


### Missing Values

Interesting.  `Combined_Key` could take care of any NaN's that we have in `Admin2`, but that could lead to some issues later if we want to consolidate the data on a state-by-state basis.  Let's check for NaN values in the first few columns.

In [19]:
# Broad stroke check of the first few columns
for i in range(0, 4):
    print(jhu_df.columns[i])
    print(jhu_df.iloc[:,i].isna().any())
    print('----------------------------------------')

iso3
False
----------------------------------------
Admin2
True
----------------------------------------
Province_State
False
----------------------------------------
Combined_Key
False
----------------------------------------


As suspected, `Admin2` is missing some values.  Let's check to make sure it's just the territories.

In [20]:
missing = jhu_df[jhu_df['Admin2'].isna() == True]

# Viewing first 20 rows
missing.head(20)

,iso3,Admin2,Province_State,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20
100,ASM,NaN,American Samoa,"American Samoa, US",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
336,USA,NaN,Diamond Princess,"Diamond Princess, US",0,0,0,0,0,0,...,49,49,49,49,49,49,49,49,49,49
570,USA,NaN,Grand Princess,"Grand Princess, US",0,0,0,0,0,0,...,103,103,103,103,103,103,103,103,103,103
571,GUM,NaN,Guam,"Guam, US",0,0,0,0,0,0,...,2074,2074,2147,2190,2235,2263,2286,2286,2286,2286
2121,MNP,NaN,Northern Mariana Islands,"Northern Mariana Islands, US",0,0,0,0,0,0,...,68,68,69,69,69,69,69,70,70,70
3007,VIR,NaN,Virgin Islands,"Virgin Islands, US",0,0,0,0,0,0,...,1257,1269,1269,1278,1290,1290,1296,1317,1317,1318


In [21]:
missing['Admin2'].isna()

100     True
336     True
570     True
571     True
2121    True
3007    True
Name: Admin2, dtype: bool

There are **only 6 missing values** and, as we thought, they are **all territories**.  We'll impute these values with the name of the territories.

In [22]:
jhu_df['Admin2'].fillna(jhu_df['Province_State'], inplace=True)

# Checking
jhu_df.Admin2.isna().sum()

0

In [23]:
jhu_df.Admin2.value_counts()

Unassigned    52
Washington    31
Jefferson     26
Franklin      25
Lincoln       24
              ..
Lenawee        1
Culberson      1
Orangeburg     1
Antrim         1
Eaton          1
Name: Admin2, Length: 1984, dtype: int64

### Placeholders
It appears that there is a placeholder value designated as `Unassigned`.  Time to figure out what these values might point to.

In [24]:
placeholder = jhu_df[jhu_df['Admin2'] == 'Unassigned']

In [25]:
placeholder.head(52)

,iso3,Admin2,Province_State,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20
64,USA,Unassigned,Alabama,"Unassigned, Alabama, US",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95,USA,Unassigned,Alaska,"Unassigned, Alaska, US",0,0,0,0,0,0,...,1,1,1,1,17,17,18,19,20,20
115,USA,Unassigned,Arizona,"Unassigned, Arizona, US",0,0,0,0,0,0,...,3,4,1,1,0,3,0,2,0,1
188,USA,Unassigned,Arkansas,"Unassigned, Arkansas, US",0,0,0,0,0,0,...,1649,1691,1686,1592,1624,1778,1766,1857,1772,1900
251,USA,Unassigned,California,"Unassigned, California, US",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
317,USA,Unassigned,Colorado,"Unassigned, Colorado, US",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
329,USA,Unassigned,Connecticut,"Unassigned, Connecticut, US",0,0,0,0,0,0,...,121,121,142,137,135,130,132,132,132,147
335,USA,Unassigned,Delaware,"Unassigned, Delaware, US",0,0,0,0,0,0,...,476,480,480,482,482,488,487,489,489,493
339,USA,Unassigned,District of Columbia,"Unassigned, District of Columbia, US",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
403,USA,Unassigned,Florida,"Unassigned, Florida, US",0,0,0,0,0,0,...,1592,1598,1600,1613,1623,1645,1654,1669,1687,1690


In [26]:
placeholder.Province_State.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Puerto Rico', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [27]:
# Checking to see if Washington D.C. is treated as its own 'state' in the 
# parent dataset
jhu_df.Province_State.unique()

array(['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'Diamond Princess', 'District of Columbia', 'Florida', 'Georgia',
       'Grand Princess', 'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Northern Mariana Islands', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
       'Vermont', 'Virgin Islands', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

So there seems to be an `Unassigned` value attached to every state, Puerto Rico, and a separate one for Washington D.C..  We also see that the Washington D.C. is being reported as if it was its own state. 

After some research, I've discovered that this was a solution by Johns Hopkins University to account for changes in how states reported their daily statistics.  Whether it was a state no longer reporting data on county levels or redefining "probable" cases. Since there is still data in these fields, I don't want to drop these.  Instead, we'll leave them be.

### Finish Preprocessing & Export

Let's move on with preprocessing this data.  We'll drop `iso3` and `Combined_Key` since they're redundant.

In [28]:
jhu = jhu_df.drop(['iso3', 'Combined_Key'], axis=1)
jhu.head()

,Admin2,Province_State,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20
0,Autauga,Alabama,0,0,0,0,0,0,0,0,...,1673,1690,1691,1714,1715,1738,1757,1764,1773,1785
1,Baldwin,Alabama,0,0,0,0,0,0,0,0,...,5047,5061,5087,5124,5141,5165,5456,5477,5526,5588
2,Barbour,Alabama,0,0,0,0,0,0,0,0,...,830,835,838,848,851,857,873,882,885,886
3,Bibb,Alabama,0,0,0,0,0,0,0,0,...,628,632,636,635,638,642,652,654,656,657
4,Blount,Alabama,0,0,0,0,0,0,0,0,...,1542,1551,1560,1573,1580,1594,1608,1611,1617,1618


We'll save swapping the axes for when it's time to start modeling. So let's export this to a csv!

In [29]:
jhu.to_csv('jhu_confirmed_preprocessed')

## States_Daily

This dataset contains a lot of great information that will be useful for EDA and modeling, too!

In [30]:
states_daily = pd.read_csv('csv_files/states_daily_Oct4.csv')
states_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11970 entries, 0 to 11969
Data columns (total 54 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   date                         11970 non-null  int64  
 1   state                        11970 non-null  object 
 2   positive                     11871 non-null  float64
 3   negative                     11733 non-null  float64
 4   pending                      1364 non-null   float64
 5   totalTestResults             11959 non-null  float64
 6   hospitalizedCurrently        9123 non-null   float64
 7   hospitalizedCumulative       6813 non-null   float64
 8   inIcuCurrently               5169 non-null   float64
 9   inIcuCumulative              1953 non-null   float64
 10  onVentilatorCurrently        4315 non-null   float64
 11  onVentilatorCumulative       682 non-null    float64
 12  recovered                    8198 non-null   float64
 13  dataQualityGrade

In [31]:
state_df = states_daily[['date', 'state', 'positiveIncrease', 
                         'negativeIncrease', 'total', 
                         'totalTestResultsIncrease']]
state_df.head()

,date,state,positiveIncrease,negativeIncrease,total,totalTestResultsIncrease
0,20201003,AK,147,3819,473256,3966
1,20201003,AL,1682,9707,1174699,10444
2,20201003,AR,746,9936,1068412,10478
3,20201003,AS,0,0,1616,0
4,20201003,AZ,636,9040,1492856,9641


In [32]:
state_df['date'] = pd.to_datetime(state_df['date'], format='%Y%m%d')
state_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11970 entries, 0 to 11969
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   date                      11970 non-null  datetime64[ns]
 1   state                     11970 non-null  object        
 2   positiveIncrease          11970 non-null  int64         
 3   negativeIncrease          11970 non-null  int64         
 4   total                     11970 non-null  int64         
 5   totalTestResultsIncrease  11970 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 561.2+ KB


In [33]:
state_df.to_csv('states_daily_preprocessed.csv')

# Jhu_Confirmed_Folium2

This data will allow us to use the FIPS codes of each county and plot this information on a map.

**Note**: The columns "apr_totals", "may_totals", "june_totals", "july_totals", and "aug_totals" were made in using Microsoft Excel.  Although, I **could have utilized for loops and `df.iloc[x:x]`** to get the same results, I **chose to use Excel for efficiency**. 

In [34]:
county = pd.read_csv('jhu_confirmed_folium2.csv')
county.head()

,UID,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/2020,1/23/2020,...,8/23/2020,8/24/2020,8/25/2020,8/26/2020,8/27/2020,8/28/2020,8/29/2020,8/30/2020,8/31/2020,aug_totals
0,84001001,1001.0,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",0,0,...,1266,1286,1286,1281,1284,1296,1309,1345,1348,76029
1,84001003,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",0,0,...,4069,4083,4100,4166,4199,4230,4307,4402,4422,193862
2,84001005,1005.0,Barbour,Alabama,US,31.868263,-85.387129,"Barbour, Alabama, US",0,0,...,629,633,628,616,614,620,624,628,628,42515
3,84001007,1007.0,Bibb,Alabama,US,32.996421,-87.125115,"Bibb, Alabama, US",0,0,...,510,510,511,513,515,517,523,533,535,27805
4,84001009,1009.0,Blount,Alabama,US,33.982109,-86.567906,"Blount, Alabama, US",0,0,...,934,946,967,968,979,983,1004,1025,1034,45132


Right off of the bat, I see two major issues.

1. FIPS codes are floats instead of integers
2. FIPS codes are supposed to be 5 digits 

In [35]:
# Rename Admin2 to County
county = county.rename(columns={'Admin2':'county'})

In [36]:
#Checking for NaNs
county.isna().any()

UID               False
FIPS               True
county             True
Province_State    False
Country_Region    False
                  ...  
8/28/2020         False
8/29/2020         False
8/30/2020         False
8/31/2020         False
aug_totals        False
Length: 235, dtype: bool

In [37]:
# County['county'] NaNs

missing_county = county[county['county'].isna() == True]
missing_county.head(10)

,UID,FIPS,county,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/2020,1/23/2020,...,8/23/2020,8/24/2020,8/25/2020,8/26/2020,8/27/2020,8/28/2020,8/29/2020,8/30/2020,8/31/2020,aug_totals
100,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,"American Samoa, US",0,0,...,0,0,0,0,0,0,0,0,0,0
336,84088888,88888.0,NaN,Diamond Princess,US,0.0000,0.0000,"Diamond Princess, US",0,0,...,49,49,49,49,49,49,49,49,49,8226
570,84099999,99999.0,NaN,Grand Princess,US,0.0000,0.0000,"Grand Princess, US",0,0,...,103,103,103,103,103,103,103,103,103,16471
571,316,66.0,NaN,Guam,US,13.4443,144.7937,"Guam, US",0,0,...,767,907,984,1120,1232,1287,1287,1287,1395,46168
2121,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,"Northern Mariana Islands, US",0,0,...,54,54,54,54,54,54,56,56,56,4486
3007,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,"Virgin Islands, US",0,0,...,980,984,998,1030,1052,1075,1118,1129,1134,37064


These are all territories so we'll do what we did before and fill the missing values with that territory's name.

In [38]:
county['county'].fillna(county['Province_State'], inplace=True)
county['county'].isna().any()

False

Now to check the missing FIPS codes, as those are vital to this dataset.

In [39]:
fips = county[county['FIPS'].isna() == True]
fips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 1267 to 2990
Columns: 235 entries, UID to aug_totals
dtypes: float64(3), int64(228), object(4)
memory usage: 18.4+ KB


In [40]:
fips.head(10)

,UID,FIPS,county,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/2020,1/23/2020,...,8/23/2020,8/24/2020,8/25/2020,8/26/2020,8/27/2020,8/28/2020,8/29/2020,8/30/2020,8/31/2020,aug_totals
1267,84070002,NaN,Dukes and Nantucket,Massachusetts,US,41.406747,-70.687635,"Dukes and Nantucket,Massachusetts,US",0,0,...,86,87,92,92,92,95,97,97,97,9387
1304,84070005,NaN,Federal Correctional Institution (FCI),Michigan,US,0.000000,0.000000,"Federal Correctional Institution (FCI), Michig...",0,0,...,195,195,195,195,195,195,195,196,196,21527
1336,84070004,NaN,Michigan Department of Corrections (MDOC),Michigan,US,0.000000,0.000000,"Michigan Department of Corrections (MDOC), Mic...",0,0,...,4907,4963,5163,5164,5185,5189,5189,5276,5284,495937
1591,84070003,NaN,Kansas City,Missouri,US,39.099700,-94.578600,"Kansas City,Missouri,US",0,0,...,8422,8509,8538,8686,8837,8995,9109,9183,9295,441355
2954,84070015,NaN,Bear River,Utah,US,41.521068,-113.083282,"Bear River, Utah, US",0,0,...,2482,2491,2500,2510,2526,2536,2550,2568,2577,165234
2959,84070016,NaN,Central Utah,Utah,US,39.372319,-111.575868,"Central Utah, Utah, US",0,0,...,484,486,490,494,498,503,506,510,511,26356
2978,84070017,NaN,Southeast Utah,Utah,US,38.996171,-110.701396,"Southeast Utah, Utah, US",0,0,...,128,133,134,135,137,139,141,142,143,6921
2979,84070018,NaN,Southwest Utah,Utah,US,37.854472,-111.441876,"Southwest Utah, Utah, US",0,0,...,3454,3463,3485,3504,3522,3541,3565,3583,3598,207555
2982,84070019,NaN,TriCounty,Utah,US,40.124915,-109.517442,"TriCounty, Utah, US",0,0,...,197,198,201,201,203,204,204,207,207,10347
2990,84070020,NaN,Weber-Morgan,Utah,US,41.271160,-111.914512,"Weber-Morgan, Utah, US",0,0,...,3150,3162,3190,3223,3243,3273,3310,3336,3346,175381


10 missing values:  
* 2 are **related to a federal and state prison in Michigan**; FCI is in Washtenaw County, while the other is the **report of all correctional facilities** in the state.
* 1 is the **cumulative result**s of both **Dukes** and **Nantucket** counties in Massachusetts
* 1 for **Kansas City, MO**, a city **split amongst 4 different counties**
    * Jackson, Cass, Clay, and Platte
* 6 entries are reports from **separate health departments within Utah** that cover multiple counties.
    * Bear River: Box Elder, Cache, and Rich
    * Central UT: Juab, Millard, Piute, Sanpete, Sevier, and Wayne
    * Southeast UT: Carbon, Emery, and Grand
    * Souhtwest UT: Beaver, Garfield, Iron, Kane, and Washington 
    * Weber-Morgan UT: Weber and Morgan

Let's double check our dataset for these counties. For now, we'll leave out the prison data.    

In [41]:
county_list = ['Jackson', 'Cass', 'Clay', 'Platte', 'Box Elder', 'Cache', 
               'Rich', 'Juab', 'Millard', 'Piute', 'Sanpete', 'Sevier', 
               'Wayne', 'Carbon', 'Emery', 'Grand', 'Beaver', 'Garfield', 
               'Iron', 'Kane', 'Washington', 'Weber', 'Morgan', 'Dukes', 
               'Nantucket']

county_check = county[county['county'].str.startswith(tuple(county_list))]
county_check

,UID,FIPS,county,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/2020,1/23/2020,...,8/23/2020,8/24/2020,8/25/2020,8/26/2020,8/27/2020,8/28/2020,8/29/2020,8/30/2020,8/31/2020,aug_totals
13,84001027,1027.0,Clay,Alabama,US,33.269842,-85.858361,"Clay, Alabama, US",0,0,...,316,317,317,333,329,338,342,352,356,14550
35,84001071,1071.0,Jackson,Alabama,US,34.781442,-85.997505,"Jackson, Alabama, US",0,0,...,1216,1217,1223,1238,1237,1247,1258,1274,1275,55821
51,84001103,1103.0,Morgan,Alabama,US,34.455006,-86.854759,"Morgan, Alabama, US",0,0,...,2620,2629,2638,2698,2723,2742,2777,2802,2811,148753
66,84001129,1129.0,Washington,Alabama,US,31.409279,-88.206899,"Washington, Alabama, US",0,0,...,484,484,487,494,492,492,495,498,502,24099
128,84005021,5021.0,Clay,Arkansas,US,36.368262,-90.414817,"Clay, Arkansas, US",0,0,...,164,164,165,167,170,172,175,180,182,8054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3267,84055053,55053.0,Jackson,Wisconsin,US,44.318649,-90.805668,"Jackson, Wisconsin, US",0,0,...,71,71,71,72,74,75,76,76,76,4395
3294,84055103,55103.0,Richland,Wisconsin,US,43.376278,-90.430127,"Richland, Wisconsin, US",0,0,...,41,41,42,44,44,46,47,49,49,2810
3309,84055131,55131.0,Washington,Wisconsin,US,43.368637,-88.229747,"Washington, Wisconsin, US",0,0,...,1458,1460,1482,1521,1543,1565,1595,1622,1622,72061
3318,84056007,56007.0,Carbon,Wyoming,US,41.693578,-106.932608,"Carbon, Wyoming, US",0,0,...,184,185,185,185,185,185,190,190,191,6682


In [42]:
# filter by state
state = ['Missouri', 'Massachusetts', 'Utah']

state_check = county_check[county_check['Province_State'].str
                            .startswith(tuple(state))]
state_check

,UID,FIPS,county,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/2020,1/23/2020,...,8/23/2020,8/24/2020,8/25/2020,8/26/2020,8/27/2020,8/28/2020,8/29/2020,8/30/2020,8/31/2020,aug_totals
1266,84025007,25007.0,Dukes,Massachusetts,US,41.406747,-70.687635,"Dukes, Massachusetts, US",0,0,...,0,0,0,0,0,0,0,0,0,0
1267,84070002,NaN,Dukes and Nantucket,Massachusetts,US,41.406747,-70.687635,"Dukes and Nantucket,Massachusetts,US",0,0,...,86,87,92,92,92,95,97,97,97,9387
1273,84025019,25019.0,Nantucket,Massachusetts,US,41.294202,-70.087747,"Nantucket, Massachusetts, US",0,0,...,0,0,0,0,0,0,0,0,0,0
1558,84029037,29037.0,Cass,Missouri,US,38.647894,-94.353399,"Cass, Missouri, US",0,0,...,961,971,975,998,1009,1027,1053,1065,1075,42408
1563,84029047,29047.0,Clay,Missouri,US,39.311321,-94.418509,"Clay, Missouri, US",0,0,...,1261,1269,1285,1309,1323,1346,1355,1363,1381,68822
1586,84029093,29093.0,Iron,Missouri,US,37.554879,-90.773530,"Iron, Missouri, US",0,0,...,40,40,42,43,44,47,47,48,49,1262
1587,84029095,29095.0,Jackson,Missouri,US,39.010022,-94.347245,"Jackson, Missouri, US",0,0,...,5038,5071,5116,5184,5254,5324,5386,5462,5521,246858
1611,84029141,29141.0,Morgan,Missouri,US,38.418177,-92.887919,"Morgan, Missouri, US",0,0,...,108,108,109,113,114,117,117,118,120,4531
1624,84029165,29165.0,Platte,Missouri,US,39.381544,-94.770685,"Platte, Missouri, US",0,0,...,441,442,448,454,460,465,472,478,477,23508
1653,84029221,29221.0,Washington,Missouri,US,37.961301,-90.879482,"Washington, Missouri, US",0,0,...,191,199,200,212,224,227,228,237,238,6189


## Summary of Missing Data Investigation:

Here is what we know:
* **All of the counties** under the umbrella of individual health departments **are accounted for** in the dataset

* Johns Hopkins University works diligently with state and county health departments to ensure that the data they collect is distributed to their respective locations and dates. The same can also be said for the data they collected from the two prisons in Michigan.  Even when spikes occur due to backlogging, **JHU ensures that everything gets properly distributed** to the proper dates so that their data remains accurate.  

Normally, I hate to drop data.  However, due to the fact that **all counties are accounted for**, and that **Johns Hopkins University** has such a **good reputation for reliable data**, I feel it is best to **throwout the missing data** here.  We have **no evidence** that the people at JHU have not already distributed the data accordingly, and to do that ourselves could **possibly inflate the numbers**.  

**Conclusion**: Drop the rows that have NaN values in the FIPS column.

In [43]:
county.dropna(how='any', inplace=True)
county.isna().any()

UID               False
FIPS              False
county            False
Province_State    False
Country_Region    False
                  ...  
8/28/2020         False
8/29/2020         False
8/30/2020         False
8/31/2020         False
aug_totals        False
Length: 235, dtype: bool

In [44]:
county

,UID,FIPS,county,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/2020,1/23/2020,...,8/23/2020,8/24/2020,8/25/2020,8/26/2020,8/27/2020,8/28/2020,8/29/2020,8/30/2020,8/31/2020,aug_totals
0,84001001,1001.0,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",0,0,...,1266,1286,1286,1281,1284,1296,1309,1345,1348,76029
1,84001003,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",0,0,...,4069,4083,4100,4166,4199,4230,4307,4402,4422,193862
2,84001005,1005.0,Barbour,Alabama,US,31.868263,-85.387129,"Barbour, Alabama, US",0,0,...,629,633,628,616,614,620,624,628,628,42515
3,84001007,1007.0,Bibb,Alabama,US,32.996421,-87.125115,"Bibb, Alabama, US",0,0,...,510,510,511,513,515,517,523,533,535,27805
4,84001009,1009.0,Blount,Alabama,US,33.982109,-86.567906,"Blount, Alabama, US",0,0,...,934,946,967,968,979,983,1004,1025,1034,45132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335,84056039,56039.0,Teton,Wyoming,US,43.935225,-110.589080,"Teton, Wyoming, US",0,0,...,409,409,412,417,417,424,425,429,430,26724
3336,84056041,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,"Uinta, Wyoming, US",0,0,...,283,283,283,289,289,292,292,295,298,18440
3337,84090056,90056.0,Unassigned,Wyoming,US,0.000000,0.000000,"Unassigned, Wyoming, US",0,0,...,0,0,0,0,53,53,73,73,73,326
3338,84056043,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,"Washakie, Wyoming, US",0,0,...,108,108,108,108,108,108,108,108,108,5937


In [45]:
# Change FIPS to int data type
county['FIPS'] = county.FIPS.astype(int)
type(county['FIPS'][0])

numpy.int32

## County Placeholders
An unsettling discovery is the value for `county` above in line 3337 labeled as `Unassigned`. We'll look into that now.   

In [46]:
# Checking the Unique values of counties for any placeholders
unassigned = county[county['county'] == 'Unassigned']
unassigned.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 52 entries, 64 to 3337
Columns: 235 entries, UID to aug_totals
dtypes: float64(2), int32(1), int64(228), object(4)
memory usage: 95.7+ KB


In [47]:
# Checking 
unassigned.head(50)

,UID,FIPS,county,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/2020,1/23/2020,...,8/23/2020,8/24/2020,8/25/2020,8/26/2020,8/27/2020,8/28/2020,8/29/2020,8/30/2020,8/31/2020,aug_totals
64,84090001,90001,Unassigned,Alabama,US,0.0,0.0,"Unassigned, Alabama, US",0,0,...,4966,5941,6288,7128,8229,8462,8605,8779,8906,198339
95,84090002,90002,Unassigned,Alaska,US,0.0,0.0,"Unassigned, Alaska, US",0,0,...,1,2,2,2,3,4,2,1,1,49
115,84090004,90004,Unassigned,Arizona,US,0.0,0.0,"Unassigned, Arizona, US",0,0,...,1,1,1,0,0,0,0,0,0,18
188,84090005,90005,Unassigned,Arkansas,US,0.0,0.0,"Unassigned, Arkansas, US",0,0,...,810,826,820,794,806,815,901,904,904,95117
251,84090006,90006,Unassigned,California,US,0.0,0.0,"Unassigned, California, US",0,0,...,0,0,0,0,0,0,0,0,0,285
317,84090008,90008,Unassigned,Colorado,US,0.0,0.0,"Unassigned, Colorado, US",0,0,...,0,0,0,0,0,0,0,0,0,8742
329,84090009,90009,Unassigned,Connecticut,US,0.0,0.0,"Unassigned, Connecticut, US",0,0,...,214,234,226,221,204,104,104,104,110,42892
335,84090010,90010,Unassigned,Delaware,US,0.0,0.0,"Unassigned, Delaware, US",0,0,...,273,275,277,276,275,279,278,279,285,14487
339,84090011,90011,Unassigned,District of Columbia,US,0.0,0.0,"Unassigned, District of Columbia, US",0,0,...,0,0,0,0,0,0,0,0,0,0
403,84090012,90012,Unassigned,Florida,US,0.0,0.0,"Unassigned, Florida, US",0,0,...,838,851,862,886,911,959,984,998,1000,40369


We have a dilemma here.  According to JHU's GitHub page, there are a few different reasons for why they have the value `Unassigned` [https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data#data-modification-records].  One reason could be **changes in a state's definition of "probable cases"**, or reporting methods.  It is also **used as a placeholder** when they receive a **backlogged reports** and they need to hold onto it until they can properly distribute them.  The label has also been used as a **placeholder for probable cases** in general, before JHU made the decision to include that data with the confirmed cases.  

Once again, we're left with a tough decision.  Before we make it though, let's just check one more time for placeholders.  It appears that JHU likes to make placeholders for every state.  We'll **subset the dataframe** on the state level and **check for unique counties** again. 

In [48]:
# Creating a New York subset
placeholder = county[county['Province_State'] == 'New York']

# Checking for unique county values
placeholder.county.unique()

array(['Albany', 'Allegany', 'Bronx', 'Broome', 'Cattaraugus', 'Cayuga',
       'Chautauqua', 'Chemung', 'Chenango', 'Clinton', 'Columbia',
       'Cortland', 'Delaware', 'Dutchess', 'Erie', 'Essex', 'Franklin',
       'Fulton', 'Genesee', 'Greene', 'Hamilton', 'Herkimer', 'Jefferson',
       'Kings', 'Lewis', 'Livingston', 'Madison', 'Monroe', 'Montgomery',
       'Nassau', 'New York', 'Niagara', 'Oneida', 'Onondaga', 'Ontario',
       'Orange', 'Orleans', 'Oswego', 'Otsego', 'Out of NY', 'Putnam',
       'Queens', 'Rensselaer', 'Richmond', 'Rockland', 'Saratoga',
       'Schenectady', 'Schoharie', 'Schuyler', 'Seneca', 'St. Lawrence',
       'Steuben', 'Suffolk', 'Sullivan', 'Tioga', 'Tompkins', 'Ulster',
       'Unassigned', 'Warren', 'Washington', 'Wayne', 'Westchester',
       'Wyoming', 'Yates'], dtype=object)

And there it is! `Out of NY`. Now to check how many there are of them.

In [49]:
out = county[county['county'].str.startswith('Out of')]
out

,UID,FIPS,county,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/2020,1/23/2020,...,8/23/2020,8/24/2020,8/25/2020,8/26/2020,8/27/2020,8/28/2020,8/29/2020,8/30/2020,8/31/2020,aug_totals
52,84080001,80001,Out of AL,Alabama,US,0.0,0.0,"Out of AL, Alabama, US",0,0,...,0,0,0,0,0,0,0,0,0,0
89,84080002,80002,Out of AK,Alaska,US,0.0,0.0,"Out of AK, Alaska, US",0,0,...,0,0,0,0,0,0,0,0,0,0
111,84080004,80004,Out of AZ,Arizona,US,0.0,0.0,"Out of AZ, Arizona, US",0,0,...,0,0,0,0,0,0,0,0,0,0
170,84080005,80005,Out of AR,Arkansas,US,0.0,0.0,"Out of AR, Arkansas, US",0,0,...,0,0,0,0,0,0,0,0,0,0
225,84080006,80006,Out of CA,California,US,0.0,0.0,"Out of CA, California, US",0,0,...,0,0,0,0,0,0,0,0,0,0
302,84080008,80008,Out of CO,Colorado,US,0.0,0.0,"Out of CO, Colorado, US",0,0,...,0,0,0,0,0,0,0,0,0,0
327,84080009,80009,Out of CT,Connecticut,US,0.0,0.0,"Out of CT, Connecticut, US",0,0,...,0,0,0,0,0,0,0,0,0,0
333,84080010,80010,Out of DE,Delaware,US,0.0,0.0,"Out of DE, Delaware, US",0,0,...,0,0,0,0,0,0,0,0,0,0
338,84080011,80011,Out of DC,District of Columbia,US,0.0,0.0,"Out of DC, District of Columbia, US",0,0,...,0,0,0,0,0,0,0,0,0,0
389,84080012,80012,Out of FL,Florida,US,0.0,0.0,"Out of FL, Florida, US",0,0,...,0,0,0,0,0,0,0,0,0,0


As we thought, there is **one for every state**.  This placeholder is either for residents of a state who were tested in a different state and/or contracted the disease outside of their state, or vice versa. 

The interesting thing is that **both of the placeholders we've found contain FIPS codes**, which we'll need for our maps.  Let's **keep these for now** and see what happens when we start mapping.

## Incomplete FIPS Codes

At the beginning I mentioned that the FIPS codes were too short, so let's take a look at them again.

In [50]:
county

,UID,FIPS,county,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/2020,1/23/2020,...,8/23/2020,8/24/2020,8/25/2020,8/26/2020,8/27/2020,8/28/2020,8/29/2020,8/30/2020,8/31/2020,aug_totals
0,84001001,1001,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",0,0,...,1266,1286,1286,1281,1284,1296,1309,1345,1348,76029
1,84001003,1003,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",0,0,...,4069,4083,4100,4166,4199,4230,4307,4402,4422,193862
2,84001005,1005,Barbour,Alabama,US,31.868263,-85.387129,"Barbour, Alabama, US",0,0,...,629,633,628,616,614,620,624,628,628,42515
3,84001007,1007,Bibb,Alabama,US,32.996421,-87.125115,"Bibb, Alabama, US",0,0,...,510,510,511,513,515,517,523,533,535,27805
4,84001009,1009,Blount,Alabama,US,33.982109,-86.567906,"Blount, Alabama, US",0,0,...,934,946,967,968,979,983,1004,1025,1034,45132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335,84056039,56039,Teton,Wyoming,US,43.935225,-110.589080,"Teton, Wyoming, US",0,0,...,409,409,412,417,417,424,425,429,430,26724
3336,84056041,56041,Uinta,Wyoming,US,41.287818,-110.547578,"Uinta, Wyoming, US",0,0,...,283,283,283,289,289,292,292,295,298,18440
3337,84090056,90056,Unassigned,Wyoming,US,0.000000,0.000000,"Unassigned, Wyoming, US",0,0,...,0,0,0,0,53,53,73,73,73,326
3338,84056043,56043,Washakie,Wyoming,US,43.904516,-107.680187,"Washakie, Wyoming, US",0,0,...,108,108,108,108,108,108,108,108,108,5937


Ok, so it's not true for all states.  FIPS are organized in such a way where the first two digits represent the state and start at "01", not "1".  I've looked at the raw file on the GitHub for this dataset and the values were saved as floats.  **Python doesn't allow integers or floats to be stored with a "0" at the beginning**.  

To overcome this, we'll convert the column values to a string and then utilize `zfill` to **impute our missing numbers** by setting the parameter to 5.

In [51]:
county['FIPS'] = county['FIPS'].astype(str)


In [52]:
county['FIPS'] = county['FIPS'].apply(lambda x: x.zfill(5))

In [53]:
county.head()

,UID,FIPS,county,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/2020,1/23/2020,...,8/23/2020,8/24/2020,8/25/2020,8/26/2020,8/27/2020,8/28/2020,8/29/2020,8/30/2020,8/31/2020,aug_totals
0,84001001,01001,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",0,0,...,1266,1286,1286,1281,1284,1296,1309,1345,1348,76029
1,84001003,01003,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",0,0,...,4069,4083,4100,4166,4199,4230,4307,4402,4422,193862
2,84001005,01005,Barbour,Alabama,US,31.868263,-85.387129,"Barbour, Alabama, US",0,0,...,629,633,628,616,614,620,624,628,628,42515
3,84001007,01007,Bibb,Alabama,US,32.996421,-87.125115,"Bibb, Alabama, US",0,0,...,510,510,511,513,515,517,523,533,535,27805
4,84001009,01009,Blount,Alabama,US,33.982109,-86.567906,"Blount, Alabama, US",0,0,...,934,946,967,968,979,983,1004,1025,1034,45132


Finally, this is preprocessed and ready to go!

In [54]:
county.to_csv('county_data_preprocessed.csv')

Time to move on to some EDA!